# Explore an FMRC using Icechunk 
This SYFEM coastal model is pushing a netCDF file to object storage each day, and also running the virtualizarr2 append_to_icechunk to update the repo. This notebook opens the up-to-date forecast model run collection and uses Rolodex to extract a best time series

In [ ]:
# load AWS credentials for Pangeo-EOSC storage as environment vars
import icechunk
import xarray as xr
from obstore.store import from_url
import os

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv(f'{os.environ['HOME']}/dotenv/school_2025.env')

username = os.environ['JUPYTERHUB_USER']

# Define storage
storage_endpoint = 'https://pangeo-eosc-minioapi.vm.fedcloud.eu'
data_bucket = 'rsignell4-protocoast'
storage_bucket = 'protocoast-school-2025'
storage_name = f'{username}-taranto'

In [ ]:
bucket = "s3://rsignell4-protocoast"
store = from_url(bucket, region="not-used", endpoint=storage_endpoint)

In [ ]:
storage = icechunk.s3_storage(
    bucket=storage_bucket,
    prefix=f"icechunk/{storage_name}",
    anonymous=True,
    endpoint_url=storage_endpoint,
    region='not-used',   # N/A for Pangeo-EOSC bucket, but required param
    force_path_style=True)

In [ ]:
config = icechunk.RepositoryConfig.default()

config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(
        url_prefix=f"s3://{storage_bucket}/",
        store=icechunk.s3_store(region="not-used", anonymous=False, s3_compatible=True, 
                                force_path_style=True, endpoint_url=storage_endpoint),
    ),
)


In [ ]:
credentials = icechunk.containers_credentials(
    {f"s3://{data_bucket}/": icechunk.s3_credentials(anonymous=False)})

read_repo = icechunk.Repository.open(
    storage, config, authorize_virtual_chunk_access=credentials)

read_session = read_repo.readonly_session("main")

In [ ]:
ds = xr.open_zarr(read_session.store, consolidated=False, zarr_format=3)
ds

In [ ]:
import rolodex.forecast
from rolodex.forecast import (
    BestEstimate,
    ConstantForecast,
    ConstantOffset,
    ForecastIndex,
    Model,
    ModelRun,
)

In [ ]:
ds.coords["valid_time"] = rolodex.forecast.create_lazy_valid_time_variable(
    reference_time=ds.time, period=ds.step
)

In [ ]:
newds = ds.drop_indexes(["time", "step"]).set_xindex(
    ["time", "step", "valid_time"], ForecastIndex)

In [ ]:
ds_best = newds.sel(valid_time=BestEstimate(offset=2))  # start at forecast hour 2 instead of 0 (analysis time)

In [ ]:
ds_best

In [ ]:
import hvplot.xarray

In [ ]:
%%time
da = ds_best['temperature'][:,100,0].compute()

In [ ]:
da.hvplot(x='valid_time', grid=True)